# CapstoneProject | Answering question # 3 :

## Does the Complaint Type that you identified in response to question 1 have an obvious relationship with any particular characteristic or characteristics of the houses or buildings?

In [23]:
import numpy as np
import pandas as pd
import types
from botocore.client import Config
import ibm_boto3
from matplotlib import pyplot as plt
import sklearn

In [2]:
def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_2a47d0a6f5c648e09ef1086c5031e6ea = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='',
    ibm_auth_endpoint="",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_2a47d0a6f5c648e09ef1086c5031e6ea.get_object(Bucket='',Key='ny_complaints.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body)

df_data_2 = pd.read_csv(body)
print('df_data_2 OK')

# @hidden_cell
body2 = client_2a47d0a6f5c648e09ef1086c5031e6ea.get_object(Bucket='',Key='pluto_19v2.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body2)
df_data_3 = pd.read_csv(body2, low_memory=False)
print('df_data_3 OK')

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (17,18,20,22,24,62,96) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## DataFrame formatting

In [4]:
df_data_2 = df_data_2[["complaint_type","incident_zip","incident_address","street_name", "latitude", "longitude"]]
print('df_data_2 dropna...')
df_data_2 = df_data_2.dropna()
print('df_data_2 is_heat_the_problem creation...')
df_data_2['is_heat_the_problem'] = np.where((df_data_2['complaint_type']=='HEAT/HOT WATER') | (df_data_2['complaint_type']=='HEATING'), True, False)
print('df_data_2 address formatting...')
df_data_2['incident_address'] = [' '.join(str(mystring).split()) for mystring in df_data_2['incident_address']]
print('df_data_3 address formatting...')
df_data_3.address = [' '.join(str(mystring).split()) for mystring in df_data_3.address]
df_data_3 = df_data_3[['borough', 'schooldist', 'address', 'numfloors', 'yearbuilt']]
print('df_data_3 dropna...')
df_data_3 = df_data_3.dropna()
print("merging...")
df_data_2 = pd.merge(df_data_2, df_data_3, how='left', left_on=['incident_address'], right_on = ['address'])
df_data_2 = df_data_2.dropna()
df_data_2['incident_zip'] = df_data_2['incident_zip'].astype('int16')
df_data_2['numfloors'] = df_data_2['numfloors'].astype("int8")
df_data_2['schooldist'] = df_data_2['schooldist'].astype("int16")
df_data_2['yearbuilt'] = df_data_2['yearbuilt'].astype('int16')
print("df_data_2 is ready ;)")

df_data_2 dropna...
df_data_2 is_heat_the_problem creation...
df_data_2 address formatting...
df_data_3 address formatting...
df_data_3 dropna...
merging...
df_data_2 is ready


In [5]:
from gc import collect
del df_data_3
collect()

710

# Finding relation between numfloors & heat complaint
### 1. Linear approach

In [28]:
df = df_data_2[df_data_2['street_name'] == "ARDEN STREET"]
df.reset_index(inplace=True)
df = df[['is_heat_the_problem', 'schooldist', 'numfloors', 'yearbuilt']]
df.head()

,is_heat_the_problem,schooldist,numfloors,yearbuilt
0,True,6,5,1930
1,True,6,5,1930
2,False,6,5,1915
3,False,6,5,1915
4,False,6,5,1915


In [22]:
result=[]
for i in range(100):    
    df = df_data_2.sample(frac=.00001)
    result.append(df['is_heat_the_problem'].corr(df['numfloors']))
print("R² max: %1.2f" % max(result))
print("R² min: %1.2f" % min(result))

R² max: 0.97
R² min: -0.95


In [52]:
from sklearn import linear_model

In [54]:
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]
regr = linear_model.LinearRegression()
x = np.asanyarray(train[['schooldist', 'numfloors', 'yearbuilt']])
y = np.asanyarray(train[['is_heat_the_problem']])
regr.fit (x, y)
# The coefficients
print ('Coefficients: ', regr.coef_)

Coefficients:  [[-0.00176701  0.00948738 -0.00011348]]


Watever the scale/sampling is, no obvious linear relation could be established between numfloors & complaint type.

Since Ŷ type is boolean, we can perform **logistic regression** on its features

### 2. Logistic Regression approach

In [24]:
from sklearn.linear_model import LogisticRegression

In [47]:
def log_solver(df, test_ratio, solver, features):
    msk = np.random.rand(len(df)) < test_ratio
    train, test = df[msk], df[~msk]
    regr = LogisticRegression(solver=solver)
    x = np.asanyarray(train[features])
    y = np.asanyarray(train[['is_heat_the_problem']]).ravel()
    regr.fit(x.reshape(-1, 1), y)
    return {'R² Coefficient':  float(regr.coef_[0]),
            "intercept": float(regr.intercept_)}


features=['schooldist', 'numfloors', 'yearbuilt']
for feat in features:
    print("with %s:" % feat, log_solver(df=df, test_ratio=0.8, solver="lbfgs", features=feat))

with schooldist: {'R² Coefficient': -0.014104776840323793, 'intercept': -0.3974953609113764}
with numfloors: {'R² Coefficient': 0.03992602549930523, 'intercept': -0.7863263216032196}
with yearbuilt: {'R² Coefficient': -0.00030600501326754573, 'intercept': -1.6027450161239406e-07}


In [43]:
sampling_sizes = [.00001, .00005, .0001, .0005, .001, .005]
results ={}
results.feature.sampling_size
iterations = 10
features = ['schooldist', 'numfloors', 'yearbuilt']
for feat in features:
    for sample_size in sampling_sizes:
        df = df_data_2.sample(frac=sample_size)
        df.reset_index(inplace=True)
        df = df[['is_heat_the_problem', 'schooldist', 'numfloors', 'yearbuilt']]
        print("data sampled at %s %% of full df, using %s:" % (100*sample_size, feat), log_solver(df=df, test_ratio=0.6, solver="lbfgs", features=feat))

data formatted at 0.001 % of full df, using schooldist: ('R² Coefficient:', array([-0.06966755]))
data formatted at 0.001 % of full df, using numfloors: ('R² Coefficient:', array([0.24927752]))
data formatted at 0.001 % of full df, using yearbuilt: ('R² Coefficient:', array([-0.000121]))
data formatted at 0.005 % of full df, using schooldist: ('R² Coefficient:', array([0.00336424]))
data formatted at 0.005 % of full df, using numfloors: ('R² Coefficient:', array([0.01873681]))
data formatted at 0.005 % of full df, using yearbuilt: ('R² Coefficient:', array([-0.00033987]))
data formatted at 0.01 % of full df, using schooldist: ('R² Coefficient:', array([-0.01650801]))
data formatted at 0.01 % of full df, using numfloors: ('R² Coefficient:', array([0.16233315]))
data formatted at 0.01 % of full df, using yearbuilt: ('R² Coefficient:', array([-0.00044756]))
data formatted at 0.05 % of full df, using schooldist: ('R² Coefficient:', array([-0.00910539]))
data formatted at 0.05 % of full df,

In [102]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, average_precision_score, jaccard_similarity_score, confusion_matrix
sampling_sizes = [.00001, .00005, .0001, .0005, .001, .005]
results ={}
#results.feature.sampling_size
iterations = 10
df_features = ['is_heat_the_problem', 'schooldist', 'numfloors', 'yearbuilt']
features = ['schooldist', 'numfloors', 'yearbuilt']
boroughs=['BX', "BK", "MN"]
def better_lr(df, features, sampling_sizes, solver="lbfgs", test_ratio=0.8, C=1000, boroughs=boroughs):
    
    results={}
    for borough in boroughs:
        results[borough]={}
        for feature in features:
            results[borough][feature] = {}
            for sample_size in sampling_sizes:
                sc = StandardScaler()
                df = df_data_2.sample(frac=sample_size)
                df = df[df['borough'] == borough]
                df.reset_index(inplace=True)
                df = df[df_features]
                df = df.astype("float64")
                #msk = np.random.rand(len(df)) < test_ratio
                #train, test = df[msk], df[~msk]
                y = np.asanyarray(df[['is_heat_the_problem']]).ravel()
                X = np.asanyarray(df[features])
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
                sc.fit(X_train)
                X_train_std = sc.transform(X_train)
                lr = LogisticRegression(C = C, random_state=0, solver=solver)
                lr.fit(X_train_std, y_train)
                #"coef": lr.coef_, 
                results[borough][feature]["sample_size %1.4f" % sample_size]={"accuracy : ": accuracy_score(y.flatten(),lr.predict(X)),
                                                                             "avg precision : ": average_precision_score(y.flatten(),lr.predict(X)),
                                                                                  "jaccard : ": jaccard_similarity_score(y.flatten(),lr.predict(X)),
                                                                              "CM": confusion_matrix(y.flatten(),lr.predict(X))
                                                                             }
    return results

results = better_lr(df=df, features=features, sampling_sizes= [.0001, .001, .01, .1], C=1000)
for key, item in results.items():
    print(key)
    for _key, _item in item.items():
        print(_key)
        for __key, __item in _item.items():
            print(__key)
            for ___key, ___item in __item.items():
                print(___key, ___item)
            print('\n')
        print('\n')
    print('\n')

BX
schooldist
sample_size 0.0001
accuracy :  0.6413793103448275
avg precision :  0.3586206896551724
jaccard :  0.6413793103448275
CM [[93  0]
 [52  0]]


sample_size 0.0010
accuracy :  0.3778580024067389
avg precision :  0.37733561774694396
jaccard :  0.3778580024067389
CM [[   2 1033]
 [   1  626]]


sample_size 0.0100
accuracy :  0.3672803398805492
avg precision :  0.36711697490578216
jaccard :  0.3672803398805492
CM [[   10 10266]
 [   10  5955]]


sample_size 0.1000
accuracy :  0.364510854866865
avg precision :  0.36419205575551933
jaccard :  0.364510854866865
CM [[   132 103301]
 [    88  59171]]




numfloors
sample_size 0.0001
accuracy :  0.6149425287356322
avg precision :  0.3850574712643678
jaccard :  0.6149425287356322
CM [[107   0]
 [ 67   0]]


sample_size 0.0010
accuracy :  0.6200378071833649
avg precision :  0.3799621928166352
jaccard :  0.6200378071833649
CM [[984   0]
 [603   0]]


sample_size 0.0100
accuracy :  0.3628746974492646
avg precision :  0.36264230950687953
ja

Results above show the method I tried could not easily predict a heat complaint, only based on numerical features like number of floors, building age or school distance. This is why I will focus on text vectorization of the address feature in the next notebook.